In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
measurements_blood_pressure = pd.read_csv('/home/bhatti/dataset/VCHAMPS/measurements_blood_pressure_train.csv').iloc[:,1:]

In [3]:
bins = [0, 60, 80, 200]
diastolic_bins = pd.crosstab(measurements_blood_pressure['Internalpatientid'], pd.cut(measurements_blood_pressure['Diastolic bp'], bins)).reset_index()

In [22]:
internalpatientid = pd.read_csv('/home/vivi/FDA_datasets/outpatient_state.csv').iloc[:,1:]['Internalpatientid'].reset_index().drop(['index'])

KeyError: "['index'] not found in axis"

In [26]:
internalpatientid.drop(columns='index',inplace=True)

In [13]:
internalpatientid = measurements_blood_pressure.groupby('Internalpatientid')['Internalpatientid']


In [28]:
internalpatientid 

,Internalpatientid
0,1
1,2
2,3
3,4
4,5
...,...
133243,169060
133244,169061
133245,169062
133246,169063


In [46]:


sys_bin = [0, 100, 130, 200]

for i in range(1,2):
    data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > i) & (measurements_blood_pressure['Age at measurement bp'] <= (i+1)*20)]
    new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Systolic bp'], bins)).reset_index().fillna(0)
    new_df = new_df.rename(columns={pd.Interval(0, 60, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20)+ ' hypotension',
                                    pd.Interval(60, 80, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' healthy',
                                    pd.Interval(80, 200, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' hypetension'})
    previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')


In [58]:
# hypotension
bins = [20,40,60,80,100, 120]
previous_df = internalpatientid 
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] < 100) | (measurements_blood_pressure['Diastolic bp'] < 60)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypotension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypotension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypotension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypotension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypotension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [68]:
# healthy
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] >= 100) & (measurements_blood_pressure['Systolic bp'] < 130) 
                                                            & (measurements_blood_pressure['Diastolic bp'] >= 60) & (measurements_blood_pressure['Diastolic bp'] < 80)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 healthy',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 healthy',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 healthy',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 healthy',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 healthy'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

KeyboardInterrupt: 

In [60]:
# hypertension
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > 130) | (measurements_blood_pressure['Diastolic bp'] > 80) ]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypertension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypertension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypertension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypertension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypertension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [69]:
previous_df

,Internalpatientid,Age 20-40 hypotension,Age 40-60 hypotension,Age 60-80 hypotension,Age 80-100 hypotension,Age 100-120 hypotension,Age 20-40 hypertension,Age 40-60 hypertension,Age 60-80 hypertension,Age 80-100 hypertension,Age 100-120 hypertension,Age 20-40 healthy,Age 40-60 healthy,Age 60-80 healthy,Age 80-100 healthy,Age 100-120 healthy
0,1,0.0,0.0,6.0,0.0,0.0,0.0,4.0,177.0,0.0,0.0,0.0,0.0,52.0,1.0,0.0
1,2,0.0,6.0,125.0,0.0,0.0,0.0,52.0,232.0,0.0,0.0,0.0,23.0,295.0,0.0,0.0
2,3,0.0,0.0,6.0,111.0,0.0,0.0,0.0,9.0,178.0,0.0,0.0,0.0,0.0,80.0,0.0
3,4,0.0,0.0,0.0,29.0,0.0,0.0,0.0,3.0,38.0,0.0,0.0,0.0,3.0,14.0,0.0
4,5,0.0,0.0,12.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133243,169060,0.0,2.0,42.0,0.0,0.0,0.0,43.0,45.0,0.0,0.0,0.0,4.0,75.0,0.0,0.0
133244,169061,0.0,0.0,10.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
133245,169062,0.0,0.0,33.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0
133246,169063,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [18]:
pd.crosstab(measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Age at measurement bp'] > 20) & (measurements_blood_pressure['Age at measurement bp'] <= 40)], pd.cut(measurements_blood_pressure['Diastolic bp'], bins)).reset_index()

Diastolic bp,Internalpatientid,"(0, 60]","(60, 80]","(80, 200]"
0,201,3,72,154
1,265,2,21,0
2,275,0,11,3
3,402,0,3,14
4,406,0,1,2
...,...,...,...,...
2743,168839,0,1,2
2744,168844,4,12,9
2745,168926,0,0,3
2746,169033,2,15,8


In [70]:
measurements_bp_preprocessed = previous_df

In [72]:
measurements_bp_preprocessed.to_csv('/home/vivi/FDA_datasets/measurements_bp_preprocessed.csv')

In [7]:
for i in range(1,2):
    data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Diastolic bp'] > i) & (measurements_blood_pressure['Age at measurement bp'] <= (i+1)*20)]
    new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Diastolic bp'], bins)).reset_index().fillna(0)
    new_df = new_df.rename(columns={pd.Interval(0, 100, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20)+ ' hypotension',
                                    pd.Interval(100, 130, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' healthy',
                                    pd.Interval(100, 130, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' hypetension'})
    previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State
0,1,61.554040,2005-10-25 00:02:08.0,75.0,140.0,Indiana
1,1,67.037260,2011-04-21 02:50:27.0,72.0,116.0,Indiana
2,1,68.304140,2012-07-27 03:54:47.0,100.0,145.0,Indiana
3,1,68.347339,2012-08-11 22:51:23.0,89.0,155.0,Indiana
4,1,68.781623,2013-01-17 16:23:39.0,72.0,143.0,Indiana
...,...,...,...,...,...,...
21997553,99999,96.324828,2013-03-28 17:23:42.0,62.0,147.0,Arizona
21997554,99999,96.326070,2013-03-29 04:17:14.0,57.0,123.0,Arizona
21997555,99999,96.331619,2013-03-31 04:57:36.0,68.0,154.0,Arizona
21997556,99999,96.357136,2013-04-09 12:47:57.0,79.0,147.0,Arizona


In [28]:
pd.Interval(1, 90, closed='right')

Interval(1, 90, closed='right')

In [29]:
diastolic_mean = measurements_blood_pressure.groupby('Internalpatientid')['Diastolic bp'].mean().reset_index()
diastolic_std = measurements_blood_pressure.groupby('Internalpatientid')['Diastolic bp'].std(ddof=0).reset_index()

In [30]:
systolic_mean = measurements_blood_pressure.groupby('Internalpatientid')['Systolic bp'].mean().reset_index()
systolic_std = measurements_blood_pressure.groupby('Internalpatientid')['Systolic bp'].std(ddof=0).reset_index()
bins = [0, 50, 60, 80, 90, 100, 300]
systolic_bins = pd.crosstab(measurements_blood_pressure['Internalpatientid'], pd.cut(measurements_blood_pressure['Systolic bp'], bins)).reset_index()

In [31]:
age_mean = measurements_blood_pressure.groupby('Internalpatientid')['Age at measurement bp'].mean().reset_index()

In [32]:
diastolic_bins = diastolic_bins.rename(columns={pd.Interval(0, 90, closed='right'): '(0,90]_diastolic',
                                                pd.Interval(90, 100, closed='right'): '(90,100]_diastolic',
                                                pd.Interval(100, 130, closed='right'): '(100,130]_diastolic',
                                                pd.Interval(130, 140, closed='right'): '(130,140]_diastolic',
                                                pd.Interval(140, 200, closed='right'): '(140,200]_diastolic'})
systolic_bins = systolic_bins.rename(columns={pd.Interval(0, 50, closed='right'): '(0,50]_systolic',
                                              pd.Interval(50, 60, closed='right'): '(50,60]_systolic',
                                              pd.Interval(60, 80, closed='right'): '(60,80]_systolic',
                                              pd.Interval(80, 90, closed='right'): '(80,90]_systolic',
                                              pd.Interval(90, 100, closed='right'): '(90,100]_systolic',
                                             
                                              pd.Interval(100, 300, closed='right'): '(100,300]_systolic'})


In [33]:
systolic_bins

Systolic bp,Internalpatientid,"(0,50]_systolic","(50,60]_systolic","(60,80]_systolic","(80,90]_systolic","(90,100]_systolic","(100,300]_systolic"
0,1,0,0,0,1,6,241
1,2,0,0,1,6,61,680
2,3,0,0,1,0,15,342
3,4,0,0,4,4,9,69
4,5,0,0,0,2,1,42
...,...,...,...,...,...,...,...
132205,169060,0,0,0,4,22,190
132206,169061,0,0,1,2,6,13
132207,169062,0,0,0,10,20,158
132208,169063,0,0,0,1,1,12


In [34]:
diastolic_mean = diastolic_mean.rename(columns={'Diastolic bp': 'diastolic mean'})
diastolic_std = diastolic_std.rename(columns={'Diastolic bp': 'diastolic std'})
systolic_mean = systolic_mean.rename(columns={'Systolic bp': 'systolic mean'})
systolic_std = systolic_std.rename(columns={'Systolic bp': 'systolic std'})
                                 

In [35]:
systolic_std 

,Internalpatientid,systolic std
0,1,16.771885
1,2,19.737801
2,3,15.850411
3,4,23.606610
4,5,18.627923
...,...,...
132205,169060,19.695581
132206,169061,21.208382
132207,169062,18.202096
132208,169063,24.597266


In [36]:
age_mean = age_mean.rename(columns={'Age at measurement bp': 'mean age at measurement bp'})

In [37]:
listTables = [diastolic_mean, diastolic_std, diastolic_bins,systolic_mean,systolic_std,systolic_bins]
measurements_bp_preprocessed = age_mean.copy()
for i in listTables:
    measurements_bp_preprocessed = measurements_bp_preprocessed.merge(i,how = 'left', on = 'Internalpatientid')


In [38]:
measurements_bp_preprocessed

,Internalpatientid,mean age at measurement bp,diastolic mean,diastolic std,"(0,90]_diastolic","(90,100]_diastolic","(100,130]_diastolic","(130,140]_diastolic","(140,200]_diastolic",systolic mean,systolic std,"(0,50]_systolic","(50,60]_systolic","(60,80]_systolic","(80,90]_systolic","(90,100]_systolic","(100,300]_systolic"
0,1,70.862416,82.629491,10.128670,197,43,8,0,0,137.832878,16.771885,0,0,0,1,6,241
1,2,65.754263,70.932157,9.821785,731,14,3,0,0,123.909792,19.737801,0,0,1,6,61,680
2,3,81.608892,65.053073,10.297528,353,2,3,0,0,129.913408,15.850411,0,0,1,0,15,342
3,4,82.307933,66.116279,12.600682,84,2,0,0,0,124.534884,23.606610,0,0,4,4,9,69
4,5,75.951072,66.022222,12.324301,44,1,0,0,0,125.977778,18.627923,0,0,0,2,1,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132205,169060,66.857634,72.962963,13.642883,189,19,8,0,0,123.694444,19.695581,0,0,0,4,22,190
132206,169061,74.858901,62.954545,10.772344,21,1,0,0,0,112.500000,21.208382,0,0,1,2,6,13
132207,169062,73.656343,76.579787,11.828518,159,25,4,0,0,119.909574,18.202096,0,0,0,10,20,158
132208,169063,74.507857,68.642857,11.787446,14,0,0,0,0,125.214286,24.597266,0,0,0,1,1,12


In [39]:
measurements_bp_preprocessed.to_csv('/home/vivi/FDA_datasets/measurements_bp_preprocessed.csv')

In [40]:
pd.isnull(measurements_bp_preprocessed).sum()

Internalpatientid             0
mean age at measurement bp    0
diastolic mean                0
diastolic std                 0
(0,90]_diastolic              0
(90,100]_diastolic            0
(100,130]_diastolic           0
(130,140]_diastolic           0
(140,200]_diastolic           0
systolic mean                 0
systolic std                  0
(0,50]_systolic               0
(50,60]_systolic              0
(60,80]_systolic              0
(80,90]_systolic              0
(90,100]_systolic             0
(100,300]_systolic            0
dtype: int64

In [41]:
measurements_bp_preprocessed[pd.isnull(measurements_bp_preprocessed['(0,50]_systolic'])]

,Internalpatientid,mean age at measurement bp,diastolic mean,diastolic std,"(0,90]_diastolic","(90,100]_diastolic","(100,130]_diastolic","(130,140]_diastolic","(140,200]_diastolic",systolic mean,systolic std,"(0,50]_systolic","(50,60]_systolic","(60,80]_systolic","(80,90]_systolic","(90,100]_systolic","(100,300]_systolic"


In [42]:
measurements_blood_pressure[measurements_blood_pressure['Internalpatientid'] == 55919]

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State
13214140,55919,74.035039,1999-01-10 19:21:28.0,105.0,220.0,Pennsylvania
